<h1>Description of Networks by Characteristics</h1>

This notebook explores the structural properties of several public transport networks in Venice by comparing them to randomly generated counterparts. 

We construct two types of random graphs: one by rewiring the original network's edges and weights, and another using the Erdős-Rényi model, both preserving key features such as node set and edge count. By analyzing metrics like average degree, shortest path length, and diameter, we assess how closely these random networks resemble the real ones. The results demonstrate that the original networks possess distinctive characteristics, significantly different from those of the random models, suggesting that their structure is not merely random or spurious.

In [1]:
import networkx as nx
import pandas as pd
import random

In [2]:
n_file=["carnival_tourist","no_carnival_tourist","carnival_residents","no_carnival_residents"]
list_Graphs_names=list()
for i in n_file:
    list_Graphs_names.append('../models/'+i+'.graphml')


## Random Graphs implementations

### Rewiring algorithm

The rewiring algorithm creates a random version of the original network by keeping the same set of nodes and redistributing the existing edge weights among randomly selected node pairs. This preserves the number of edges and the weight distribution, but randomizes the connections, allowing comparison of structural properties with the original network

### Erdos Renyi algorihtm

The Erdős-Rényi model generates random graphs by connecting pairs of nodes with edges chosen uniformly at random. In its simplest form, each possible edge between nodes is included with a fixed probability, or a fixed number of edges is distributed randomly among all possible node pairs. This produces networks with no inherent structure, serving as a baseline for comparison with real-world networks.

In [5]:
import sys
sys.path.append('../bin')

# Both algorithms are implemented in this file
import random_networks as rn

<h2>Characteristic calculation function</h2>

In [6]:
def description(list_Graphs,names):
    dict_description=dict()
    for c,G in enumerate(list_Graphs):
        i=names[c]
        dict_description[i]=list()
        dict_description[i].append(sum([d for n, d in G.degree(weight="weight")])/len([d for n, d in G.in_degree()]))      
        #The diameter and SPL require a strongly connected graph.    
        if not nx.is_strongly_connected(G):
            # Obtain the largest strongly connected component
            largest_scc = max(nx.strongly_connected_components(G), key=len)
            # Create subgraph with that component
            G = G.subgraph(largest_scc).copy()
        dict_description[i].append(nx.average_shortest_path_length(G,weight="weight"))
        dict_description[i].append(nx.diameter(G,weight="weight"))
    return dict_description

## Experiments results

### Characteristics of real networks

In [7]:
list_Graphs=list()
for i in list_Graphs_names:
    list_Graphs.append(nx.read_graphml(i))
dict_description=description(list_Graphs,n_file)
pd.DataFrame(dict_description,index=["degree_mean","avg_spl","diameter"])

,carnival_tourist,no_carnival_tourist,carnival_residents,no_carnival_residents
degree_mean,2015.555556,3267.093750,641.965517,1472.063492
avg_spl,3.473684,3.589617,2.519985,2.601110
diameter,10.000000,11.000000,9.000000,8.000000


### Characteristics of Rewired networks

The algorithm keeps each node degree, but connects nodes in a random way.

In [8]:
list_Graphs=list()
for i in list_Graphs_names:
    list_Graphs.append(rn.rewiring(nx.read_graphml(i)))
dict_description=description(list_Graphs,n_file)
pd.DataFrame(dict_description,index=["degree_mean","avg_spl","diameter"])

,carnival_tourist,no_carnival_tourist,carnival_residents,no_carnival_residents
degree_mean,2015.555556,3267.093750,641.965517,1472.063492
avg_spl,3.482335,3.550099,2.453721,2.438044
diameter,6.000000,9.000000,4.000000,5.000000


### Characteristics of Erdos Renyi networks

The algorithms rewires the network in a completely random manner

In [9]:
list_Graphs=list()
for i in list_Graphs_names:
    G=nx.read_graphml(i)
    pesos = [d['weight'] for u, v, d in G.edges(data=True) if 'weight' in d]
    # Obtener mínimo y máximo
    peso_min = min(pesos)
    peso_max = max(pesos)
    list_Graphs.append(rn.rewiring(rn.erdos_renyi_weighted_same_nodes(G, weight_range=(peso_min, peso_max), directed=True, seed=None)))
dict_description=description(list_Graphs,n_file)
pd.DataFrame(dict_description,index=["degree_mean","avg_spl","diameter"])

,carnival_tourist,no_carnival_tourist,carnival_residents,no_carnival_residents
degree_mean,62182.303817,123416.147804,40630.299037,139346.741394
avg_spl,453.042843,599.939435,708.259128,1528.638139
diameter,1192.447256,2024.694957,2093.531704,4270.640439
